# Air Sac Tracking Prototype code
Lara S. Burchardt & Wim Pouw

lara.burchardt@donders.ru.nl

Next to body movements and acoustics, we would also like to track the air sac's inflation of the Siamangs. The air sac naturally forms a spherical(3D) or circular (2D) shape, and such shapes are retrievable from an image using the hough transform, and a bit of pre-processing of the images to get the optimal representation of the relevent edges of the air sac.

This code takes as input a sample video with a close up of a Siamang, and then tracks the air sac when it takes a sufficiently circular shape. The result is shown below; it is not perfect, but with a bit of smoothing this can function as a good air sac tracker. This code is very much under development, there are many ways to improve further.

In [2]:
# import the necessary packages
import numpy as np
import argparse
import cv2
import pandas as pd
from skimage import io, feature, color, measure, draw, img_as_float
import numpy as np
import csv

#resused code from 
#https://pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
#https://stackoverflow.com/questions/31705355/how-to-detect-circlular-region-in-images-and-centre-it-with-python

## Circular Tracker

Using a video sample as input, the user first defines a region of interest in which to look for circular objects.
After a few pre-processing steps to increase tracking success, the Hough Transform is used to find circles. 
The radius and position of the circle is saved together with the used parameter combination used in pre-processing and tracking. 

In [2]:
videofolder= '../Video/'
videofilename = 'sample.mp4'
# Opens the Video file
cap = cv2.VideoCapture(videofolder+videofilename)
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)   #fps = frames per second

#output video
out = cv2.VideoWriter('./Output/output_'+videofilename,cv2.VideoWriter_fourcc(*'MP4V'), fps, 
                      (int(frameWidth), int(frameHeight)))

#set up empty output dataframe
column_names = ['frame','roi_x1','roi_y1','roi_x2', 'roi_y2', # info on region of interest for repetability
                'x','y', 'r', 'inputfile',                    # actual results 
                'alpha', 'beta',                              # values of brightness and contrast pre processing
                'dp', 'minDist', 'param1', 'param2', 'minRadius',' maxRadius'] # parameters of hough circle transform 
df = pd.DataFrame(columns = column_names)

#initialize iterator
i=0
# initialize parameters for pre-processing (used in cv2.convertScaleAbs)
alpha = 1.5
beta = 30

# ROI, define a region of interest by mouse click on the first frame
ret,frame = cap.read()
    # select ROI
roi  = cv2.selectROI('select the area', frame)
#roi = [821, 351, 612, 488]       # if you want to re run a particular roi, input the roi values here

#croppedFrame = frame[int(roi[1]): int(roi[1]+roi[3]),
#                     int(roi[0]):int(roi[0]+roi[2])] 


#main loop                      
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
          
    ############################detect circles   
    output=frame.copy()
    # transform to grayscale image only using the roi part of the image
    gray = cv2.cvtColor(frame[int(roi[1]): int(roi[1]+roi[3]),
                     int(roi[0]):int(roi[0]+roi[2])] , cv2.COLOR_BGR2GRAY)
    output_gray=gray.copy()
    # increasing brightness and contrast
    gray_light = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)
    # apply GuassianBlur to reduce noise. medianBlur is also added for smoothening, reducing noise.
    gray = cv2.GaussianBlur(gray_light,(11,11),0)
    gray = cv2.medianBlur(gray, 19)
    ##thresholded edge contouring
    hist = cv2.equalizeHist(gray)
    gamma = 2
    invGamma = 1/gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    gamm = cv2.LUT(hist, table, hist)
    submitted = cv2.Canny(gamm, 10, 50)            # why is minVal bigger than maxValue? original code was (gamm, 20, 10)
    submitted = cv2.GaussianBlur(submitted,(7,7),0)
    submitted = cv2.threshold(submitted, 25, 255, cv2.THRESH_BINARY)[1]
    submitted = cv2.dilate(submitted, None, iterations=9)  
    submitted = cv2.erode(submitted, None, iterations=10) 
    submitted = cv2.GaussianBlur(submitted,(7,7),0) 
    submitted = cv2.erode(submitted, None, iterations=4)
    
    #track demicircles 
    # define parameters for HoughTransform outside of function to be able to save and manipulate easier
    
    dp = 1
    minDist = 10000
    param1 = 10
    param2 = 10
    minRadius = 5
    maxRadius = 250
    
    # original values: dp = 1, minDist = 10000, param1=1, param2=10, minRadius = 5, maxRadius= 250
    circles = cv2.HoughCircles(submitted, cv2.HOUGH_GRADIENT, 
                               dp = dp,minDist = minDist,  
                               param1 = param1,param2 = param2, 
                               minRadius = minRadius, maxRadius = maxRadius)
    if circles is not None:
        circles = np.round(circles[0, 0:1]).astype("int")
        for (x, y, r) in circles:
            #cv2.circle(output, (x, y), r, (255, 255, 0), 2) #version without drwaing roi back on whole image
            cv2.circle(output[int(roi[1]): int(roi[1]+roi[3]),
                     int(roi[0]):int(roi[0]+roi[2])], (x, y), r, (255, 255, 0), 2)
            cv2.circle(submitted, (x, y), r, (200, 0, 0), 2)
            #print(x,y,r)
    cv2.waitKey(1)
    out.write(output)
    cv2.imshow("output", submitted)
    i=i+1

# saving results and used parameters row wise during loop into a dataframe

# canny min and max value should be added here, if they turn out to be sensitive
    new_row = [i+1, roi[0],roi[1], roi[2], roi[3], circles[0,0], circles[0,1],circles [0,2], videofilename,
               alpha, beta, dp, minDist, param1, param2, minRadius, maxRadius]
    
    df.loc[len(df)] = new_row
            
# filename and saving dataframe as cvs file       
    filename =  'airsacradius_results_' + videofilename + '.csv'
    df.to_csv(filename, sep = ',')

# cleaning up
out.release()
cap.release()
cv2.destroyAllWindows()

# some more prototyping for auto ROI

In [ ]:
# ROI, define a region of interest by mouse click on the first frame
#ret,frame = cap.read()
    # select ROI
#roi  = cv2.selectROI('select the area', frame)
#roi = [821, 351, 612, 488]       # if you want to re run a particular roi, input the roi values here

#croppedFrame = frame[int(roi[1]): int(roi[1]+roi[3]),
#                     int(roi[0]):int(roi[0]+roi[2])] 
  

In [ ]:
# To do

# 1. include timepoints in output
#    for each extracted radius, we also want to have the timepoint in the snippet to be able to correlate it with f0 as analyzed 
#    in praat or else, as we won't have the values for exactly the same timepoints 
#    for that we need the fps, that we already write, then multiply fps with frame number (should be i?) and write into output